## Harris Corner Detector with Sum of Squared Differences (SSD) and Normalized Cross Correlation (NCC) measurement metric
Tejas Pant <br>
26th September 2018

In [5]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os
import math

#Pair 1
image1=cv2.imread("../HW4Pics/pair1/1.jpg")
image2=cv2.imread("../HW4Pics/pair1/2.jpg")

'''
#Pair 2
image1=cv2.imread("../HW4Pics/pair2_resized/truck1.jpg")
image2=cv2.imread("../HW4Pics/pair2_resized/truck2.jpg")
'''

#Convert to grayscale images
image1_gray=cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
image2_gray=cv2.cvtColor(image2,cv2.COLOR_BGR2GRAY)

In [6]:
#Identify the Harris Corners from a single image
def HarrisCornerDetector(image,sigma_harris,k_harris):
    #Determine size of Haar Wavelet Filter
    haar_wav_size =int(math.ceil(math.ceil(4 * sigma_harris)/2)*2) #Greatest even integer greater than 4*sigma
    
    #Generate the Haar Wavelet Filters
    haar_wavX = np.concatenate((-np.ones((haar_wav_size,int(haar_wav_size/2))), np.ones((haar_wav_size,int(haar_wav_size/2)))),axis=1)
    haar_wavY = np.concatenate((np.ones((int(haar_wav_size/2),haar_wav_size)), -np.ones((int(haar_wav_size/2),haar_wav_size))),axis=0)

    #Filtered image after applying the Haar Operator in X and Y direction to get the gradients
    dx = cv2.filter2D(image,-1,haar_wavX) 
    dy = cv2.filter2D(image,-1,haar_wavY)
    
    #Initialize the Corner Detector Response Matrix C
    CorDetectResp = np.zeros(image.shape) 
    
    #Determine size of matrix for calculating values of the Corner Detector Response at every pixel in the image
    # Here we use the greatest odd integer greater than 5*sigma
    CHarris_size = int(math.ceil(math.ceil(5 * sigma_harris)/2)*2)+1 
    CHarris_size_half = int((CHarris_size - 1)/2)
    
    for i in range(CHarris_size_half, image.shape[0]-CHarris_size_half): #move along rows, Y direction
        for j in range(CHarris_size_half, image.shape[1]-CHarris_size_half): #move along columns, X direction
            dxij_ = dx[i-CHarris_size_half:i+CHarris_size_half+1, j-CHarris_size_half:j+CHarris_size_half+1]
            dyij_ = dy[i-CHarris_size_half:i+CHarris_size_half+1, j-CHarris_size_half:j+CHarris_size_half+1]
            Cres = np.zeros((2,2))
            Cres[0,0] = np.sum(np.square(dxij_))
            Cres[1,1] = np.sum(np.square(dyij_))
            Cres[0,1] = np.sum(np.multiply(dxij_,dyij_))
            Cres[1,0] = Cres[0,1]
            det_Cres = np.linalg.det(Cres)
            tr_Cres = np.trace(Cres);
            CorDetectResp[i,j] = det_Cres - k_harris * (tr_Cres**2)
    
    #Thresholding out pixels with small or negative values of Harris Corner Detector
    th_size = 29 #window size for thresholding the corner response value
    th_size_half = int((th_size - 1)/2) 
    corners=[]
    Cor_loc = np.zeros(image.shape)
    
    for i in range(th_size_half, image.shape[0]-th_size_half):
        for j in range(th_size_half, image.shape[1]-th_size_half):
            CHarris_sub = CorDetectResp[i-th_size_half:i+th_size_half+1,j-th_size_half:j+th_size_half+1]
            if CorDetectResp[i,j] == np.max(CHarris_sub) and CorDetectResp[i,j] > 0 and abs(CorDetectResp[i,j]) > np.mean(abs(CorDetectResp)):
                corners.append([i,j])
    return corners

#SSD Measurement Metric to establish correspondence between Interest Points
def SSD(image1,image2,cor1,cor2,ssd_win_size,ratio_global_local_min,thresh_ssd):
    ssd_winh = int((ssd_win_size-1)/2)
    ncor1 = len(cor1)
    ncor2 = len(cor2)
    int_pts=[] #interest points list
    
    #Calculate the SSD matrix for the sets of corners
    SSD_mat = np.zeros((ncor1,ncor2))
    
    for i in range(0,ncor1):
        for j in range(0,ncor2):
            img1ij_ = image1[cor1[i,0]-ssd_winh:cor1[i,0]+ssd_winh+1, cor1[i,1]-ssd_winh:cor1[i,1]+ssd_winh+1]
            img2ij_ = image2[cor2[j,0]-ssd_winh:cor2[j,0]+ssd_winh+1, cor2[j,1]-ssd_winh:cor2[j,1]+ssd_winh+1]
            diff = np.subtract(img1ij_,img2ij_)
            SSD_mat[i,j] = np.sum(np.square(diff))
    

    #Identify the corresponding corner points in the two images by thresholding 
    for i in range(0,ncor1):
        for j in range(0,ncor2):
            if SSD_mat[i,j]==np.min(SSD_mat[i,:]) and SSD_mat[i,j] < thresh_ssd * np.mean(SSD_mat[:,:]):
                min_local_value = SSD_mat[i,j]
                SSD_mat[i,j] = np.max(SSD_mat[i,:])
                if min_local_value / np.min(SSD_mat[i,:]) < ratio_global_local_min:
                    SSD_mat[:,j] = np.max(SSD_mat)
                    SSD_mat[i,j] = min_local_value
                    int_pts.append([cor1[i,0],cor1[i,1],cor2[j,0],cor2[j,1]])
    
    return np.asarray(int_pts)

#NCC Measurement Metric to establish correspondence between Interest Points
def NCC(image1,image2,cor1,cor2,ncc_win_size,thresh_ncc):
    ncc_winh = int((ncc_win_size-1)/2)
    ncor1 = len(cor1)
    ncor2 = len(cor2)
    int_pts=[] #interest points list
    
    #Calculate the NCC matrix for the sets of corners
    NCC_mat = np.zeros((ncor1,ncor2))
    
    for i in range(0,ncor1):
        for j in range(0,ncor2):
            img1ij_ = image1[cor1[i,0]-ncc_winh:cor1[i,0]+ncc_winh+1, cor1[i,1]-ncc_winh:cor1[i,1]+ncc_winh+1]
            img2ij_ = image2[cor2[j,0]-ncc_winh:cor2[j,0]+ncc_winh+1, cor2[j,1]-ncc_winh:cor2[j,1]+ncc_winh+1]
            mu1 = np.mean(img1ij_) 
            mu2 = np.mean(img2ij_)
            stddev1 = np.subtract(img1ij_,mu1)
            stddev2 = np.subtract(img2ij_,mu2)
            ncc_num = np.sum(np.multiply(stddev1,stddev2))
            var1 = np.sum(np.square(stddev1))
            var2 = np.sum(np.square(stddev2))
            ncc_den = np.sqrt(var1*var2)
            NCC_mat[i,j] = ncc_num/ncc_den

    #Identify the corresponding corner points in the two images by thresholding 
    for i in range(0,ncor1):
        for j in range(0,ncor2):
            if NCC_mat[i,j]==np.max(NCC_mat[i,:]) and NCC_mat[i,j]>thresh_ncc:
                int_pts.append([cor1[i,0],cor1[i,1],cor2[j,0],cor2[j,1]])

    return np.asarray(int_pts)

#Display combined image of the pair of images with interest points and correspondences
def displayImagewithInterestPoints(img1,img2,corners):
    #Get shape of the output image
    nrows = max(img1.shape[0], img2.shape[0])
    ncol = img1.shape[1]+img2.shape[1]
    
    #Initialize combined output image
    out_img = np.zeros((nrows,ncol,3))
    
    #Copy Image 1 to left half of the output image
    out_img[:img1.shape[0], :img1.shape[1]] = img1
    
    #Copy Image 2 to right half of the output image
    out_img[:img2.shape[0], img1.shape[1]:img1.shape[1]+img2.shape[1]] = img2

    for xy in corners:
        cv2.circle(out_img,(xy[1], xy[0]),4,(0,0,0),2) #interest points from Image 1
        cv2.circle(out_img,(img1.shape[1]+xy[3],xy[2]),4,(0,0,0),2) #Interest points form Image 2
        cv2.line(out_img,(xy[1],xy[0]),(img1.shape[1]+xy[3],xy[2]), (0,255,0)) #Lines joining interest points
    return out_img

In [7]:
#For Harris Detector
sigma_harris = 1.4
k_harris = 0.04

corner_image1 = HarrisCornerDetector(image1_gray,sigma_harris,k_harris)
corner_image2 = HarrisCornerDetector(image2_gray,sigma_harris,k_harris)

print("Number of Corners Detected in Image 1 = ", len(corner_image1))
print("Number of Corners Detected in Image 2 = ", len(corner_image2))

C:\ProgramData\Anaconda3\lib\site-packages\numpy\linalg\linalg.py:1821: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


Number of Corners Detected in Image 1 =  104
Number of Corners Detected in Image 2 =  119


In [4]:
#For SSD
ssd_ratio = 0.7 #ratio of local minima of mathces
thresh_ssd = 4.5 #threshold for mean
win_size_ssd = 25 # win_size_ssd x win_size_ssd, window size for examining ssd metric

corner_image1 = np.array(corner_image1)
corner_image2 = np.array(corner_image2)

Cord_SSD = SSD(image1_gray,image2_gray,corner_image1,corner_image2,win_size_ssd,ssd_ratio,thresh_ssd)
out_image_ssd = displayImagewithInterestPoints(image1,image2,Cord_SSD)
cv2.imwrite('HarrisSSD.jpg',out_image_ssd)

#For NCC
thresh_ncc = 0.8 #NCC threhold value. Values smaller than thresh_ncc are neglected
win_size_ncc = 25 #win_size_ncc x win_size_ncc, window size for examining ssd metric

Cord_NCC = NCC(image1_gray,image2_gray,corner_image1,corner_image2,win_size_ncc,thresh_ncc)
out_image_NCC = displayImagewithInterestPoints(image1,image2,Cord_NCC)
cv2.imwrite('HarrisNCC.jpg',out_image_NCC)

True